<a href="https://colab.research.google.com/github/kicysh/final_task_of_world_model_lecture_2021/blob/main/src/tmp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# セットアップ

- scVIのインストール
- データのダウンロード
- 必要なパッケージのインストール

In [2]:
!git clone https://github.com/tabdelaal/scVI

Cloning into 'scVI'...
remote: Enumerating objects: 10007, done.
remote: Total 10007 (delta 0), reused 0 (delta 0), pack-reused 10007
Receiving objects: 100% (10007/10007), 108.57 MiB | 37.54 MiB/s, done.
Resolving deltas: 100% (7177/7177), done.


In [10]:
!pip install /content/scVI

Processing ./scVI
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
     |████████████████████████████████| 4.8 MB 3.2 MB/s 
     |████████████████████████████████| 96 kB 4.9 MB/s 
     |████████████████████████████████| 96 kB 4.1 MB/s 
     |████████████████████████████████| 2.0 MB 51.5 MB/s 
     |████████████████████████████████| 8.8 MB 34.2 MB/s 
     |████████████████████████████████| 48 kB 4.6 MB/s 
     |████████████████████████████████| 86 kB 5.0 MB/s 
     |████████████████████████████████| 1.1 MB 62.8 MB/s 
     |████████████████████████████████| 63 kB 1.7 MB/s 
  Created wheel for scvi: filename=scvi-0.6.4-py2.py3-none-any.whl size=139

In [12]:
!mkdir data
!gsutil cp gs://h5ad/2019-02-Pijuan-Sala-et-al-Nature/pijuan_sala_atlas.h5ad /content/data

Copying gs://h5ad/2019-02-Pijuan-Sala-et-al-Nature/pijuan_sala_atlas.h5ad...
/ [1 files][  1.0 GiB/  1.0 GiB]   53.0 MiB/s                                   
Operation completed over 1 objects/1.0 GiB.                                      


In [13]:
!pip install scanpy

# LDVAEの実行

In [14]:
path_of_data = '/content/data/pijuan_sala_atlas.h5ad'

In [11]:
from scvi.dataset import AnnDatasetFromAnnData
from scvi.models import LDVAE
from scvi.inference import UnsupervisedTrainer
from scvi.inference.posterior import Posterior

/usr/local/lib/python3.7/dist-packages/numba/np/ufunc/parallel.py:363: NumbaWarning: The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 9107. The TBB threading layer is disabled.
  warnings.warn(problem)


In [15]:
import scanpy as sc
adata = sc.read_h5ad(path_of_data)
adata

AnnData object with n_obs × n_vars = 139331 × 29452
    obs: 'barcode', 'sample', 'stage', 'sequencing.batch', 'theiler', 'doub.density', 'doublet', 'cluster', 'cluster.sub', 'cluster.stage', 'cluster.theiler', 'stripped', 'celltype', 'colour', 'umapX', 'umapY', 'haem_gephiX', 'haem_gephiY', 'haem_subclust', 'endo_gephiX', 'endo_gephiY', 'endo_trajectoryName', 'endo_trajectoryDPT', 'endo_gutX', 'endo_gutY', 'endo_gutDPT', 'endo_gutCluster'
    var: 'gene_name'

In [16]:
cells_dataset = AnnDatasetFromAnnData(adata)

[2022-03-18 09:23:07,639] INFO - scvi.dataset.dataset | Remapping labels to [0,N]
[2022-03-18 09:23:07,649] INFO - scvi.dataset.dataset | Remapping batch_indices to [0,N]
[2022-03-18 09:23:25,437] INFO - scvi.dataset.dataset | Computing the library size for the new data
[2022-03-18 09:23:27,201] INFO - scvi.dataset.dataset | Downsampled from 139331 to 139331 cells


In [17]:
cells_dataset.nb_genes

29452

In [18]:
vae = LDVAE(
    cells_dataset.nb_genes,
    reconstruction_loss='nb',
    use_batch_norm=True,
    latent_distribution='normal',
    n_latent=20
)

In [19]:
vae

LDVAE(
  (z_encoder): Encoder(
    (encoder): FCLayers(
      (fc_layers): Sequential(
        (Layer 0): Sequential(
          (0): Linear(in_features=29452, out_features=128, bias=True)
          (1): BatchNorm1d(128, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          (2): ReLU()
          (3): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (mean_encoder): Linear(in_features=128, out_features=20, bias=True)
    (var_encoder): Linear(in_features=128, out_features=20, bias=True)
  )
  (l_encoder): Encoder(
    (encoder): FCLayers(
      (fc_layers): Sequential(
        (Layer 0): Sequential(
          (0): Linear(in_features=29452, out_features=128, bias=True)
          (1): BatchNorm1d(128, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          (2): ReLU()
          (3): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (mean_encoder): Linear(in_features=128, out_features=1, bias=True)
    (var_encoder): Linear(in_features

前処理  
- [Scanpyの真の力を僕達はまだ知らない](https://emoriroom.hatenablog.com/entry/2020/12/18/190000)
- [Scanpy: Preprocessing and clustering 3k PBMCs](https://singlecellanalysistutorial.readthedocs.io/en/latest/notebooks/Scanpy_PBMC.html)


In [ ]:
df = adata.to_df()

10056.1767578125

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms

In [ ]:
class Inference_model(nn.Module):
    def __init__(self, cells_cnt, genes_cnt,z_dim):
        super(Inference_model, self).__init__()
        self.cells_cnt = cells_cnt
        self.genes_cnt = genes_cnt
        self.g = nn.Sequential(nn.Linear(cells_cnt,128),
                               nn.Relu(),)
        self.g_mu = nn.Linear(128,z_dim)
        self.g_sigma = nn.Linear(128,z_dim)

    def forward(self,x):
        g_x = self.g(x)
        mu = self.g_mu(g_x)
        sigma = self.g_sigma(g_x)
        epsilon = torch.randn(mu.shape).to(device)
        return sigma + mu * epsilon


class Generative_model(nn.Module):
    def __init__(self,cells_cnt, genes_cnt,z_dim):
        super(Generative_model, self).__init__()
        self.z_dim = z_dim
        self.cells_cnt = cells_cnt
        self.genes_cnt = genes_cnt
        self.emb = nn.Embedding(k_dim,z_dim)

    def forward(self):
        pass

class LDVAE(nn.Module):
    def __init__(self, cells_cnt, genes_cnt,z_dim):
        super(LDVAE, self).__init__()
        self.z_dim = z_dim
        self.cells_cnt = cells_cnt
        self.genes_cnt = genes_cnt
        self.emb = nn.Embedding(genes_cnt,z_dim)

    def forward(self):
        pass

class scVI(nn.Module):
    def __init__(self, cells_cnt, genes_cnt, z_dim):
        super(scVI, self).__init__()
        self.cells_cnt = cells_cnt
        self.genes_cnt = genes_cnt
        self.z_dim = z_dim
        self.inference_model = Inference_model(cells_cnt, genes_cnt, z_dim)
        self.ldvae = LDVAE(cells_cnt, genes_cnt, z_dim)
        self.generative_model = Generative_model(cells_cnt,genes_cnt, z_dim)
        
    def forward(self, x):
        z = self.inference_model(x)
        mu = self.ldvae(z)

In [ ]:
figsize(7, 12)
for i in range(5):
    
    # -- Z plot --
    
    plt.subplot(5, 2, 2 * i + 1)
    
    plt.hist2d(
        Z_df_ordered[f'z{2 * i}'], Z_df_ordered[f'z{2 * i + 1}'],
        bins=256,
        norm=mcolors.PowerNorm(0.25),
        cmap=cm.gray_r,
        rasterized=True
    )
    plt.axis('equal');
    plt.xlabel(f'$Z_{2 * i}$ ({variance_explained[idx][2 * i]:.1%} variance)')
    plt.ylabel(f'$Z_{2 * i + 1}$ ({variance_explained[idx][2 * i + 1]:.1%} variance)')
    
    ax = plt.gca()
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    
    # -- W plot --
    
    plt.subplot(5, 2, 2 * i + 2)
    
    w_columns = [f'w{2 * i}', f'w{2 * i + 1}']
    
    plt.scatter(
        W_df_ordered[w_columns[0]], W_df_ordered[w_columns[1]],
        c='lightgrey',
        rasterized=True
    )
    plt.axis('equal');
    plt.xlabel(f'$W_{2 * i}$')
    plt.ylabel(f'$W_{2 * i + 1}$')
    
    tmp_ = W_df_ordered.copy()
    tmp_['lnth'] = np.linalg.norm(tmp_[w_columns], axis=1)
    
    ggg = (
        tmp_
        .sort_values('lnth', ascending=False)
        .head(8)[['gene_name', 'lnth', *w_columns]]
    )
    print(ggg['gene_name'].values)
    
    texts = []
    arrows = []
    for g, r in ggg.iterrows():
        x_, y_ = r[w_columns[0]], r[w_columns[1]]
        
        ha = 'right'
        if x_ > 0:
            ha = 'left'
            
        va = 'top'
        if y_ > 0:
            va = 'bottom'
            
        arrows.append(plt.arrow(0, 0, x_, y_, length_includes_head=True))
        
        xs, ys = 0, 0
        if (i, r.gene_name) in text_shift:
            xs, ys = text_shift[(i, r.gene_name)]
        
        texts.append(plt.text(x_ + xs, y_ + ys, r.gene_name, ha=ha, va=va))

    plt.xlim(left=-3, right=3)
    plt.ylim(bottom=-3, top=3)
        
    ax = plt.gca()
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    
plt.tight_layout()
plt.savefig('figures/linear_pij_results.pdf', bbox_inches='tight', dpi=400)

[fig.1c 実装コード](https://github.com/pachterlab/SGYP_2019/blob/master/4.a%20Pijuan-Sala%20et%20al%20result%20figures.ipynb)
